In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

## Question 1: Count Student Number in Departments

A university uses 2 data tables, student and department, to store data about its students and the departments associated with each major.

Write a query to print the respective department name and number of students majoring in each department for all departments in the department table (even ones with no current students).

Sort your results by descending number of students; if two or more departments have the same number of students, then sort those departments alphabetically by department name.

`Student`

| student_id | student_name | gender | dept_id |
|------------|--------------|--------|---------|
| 1          | Jack         | M      | 1       |
| 2          | Jane         | F      | 1       |
| 3          | Mark         | M      | 2       |

`Department`

| dept_id | dept_name   |
|---------|-------------|
| 1       | Engineering |
| 2       | Science     |
| 3       | Law         |


In [3]:
Student = pd.DataFrame({"student_id":[1,2,3],"student_name":["Jack","Jane","Mark"],
             "gender":["M","F","M"],"dept_id":[1,1,2]})
Department = pd.DataFrame({"dept_id":[1,2,3],"dept_name":["Engineering","Science","Lae"]})

In [6]:
pd.merge(Department,Student,on="dept_id",how='left').groupby("dept_name").agg({"student_name":"count"}).\
rename(columns={"student_name":"student_number"}).reset_index()

,dept_name,student_number
0,Engineering,2
1,Lae,0
2,Science,1


### Question 2: Investments in 2016

Write a query to print the sum of all total investment values in 2016 (TIV_2016), to a scale of 2 decimal places, for all policy holders who meet the following criteria:

- Have the same TIV_2015 value as one or more other policyholders.

- Are not located in the same city as any other policyholder (i.e.: the (latitude, longitude) attribute pairs must be unique).

Input Format:
The insurance table is described as follows:

where PID is the policyholder's policy ID, TIV_2015 is the total investment value in 2015, TIV_2016 is the total investment value in 2016, LAT is the latitude of the policy holder's city, and LON is the longitude of the policy holder's city.

`Insurance`

| PID | TIV_2015 | TIV_2016 | LAT | LON |
|-----|----------|----------|-----|-----|
| 1   | 10       | 5        | 10  | 10  |
| 2   | 20       | 20       | 20  | 20  |
| 3   | 10       | 30       | 20  | 20  |
| 4   | 10       | 40       | 40  | 40  |



In [22]:
Insurance = pd.DataFrame({"PID":[1,2,3,4],
             "TIV_2015":[10,20,10,10],
             "TIV_2016":[5,20,30,40],
             "LAT":[10,20,20,40],
             "LON":[10,20,20,40]})
Insurance

,PID,TIV_2015,TIV_2016,LAT,LON
0,1,10,5,10,10
1,2,20,20,20,20
2,3,10,30,20,20
3,4,10,40,40,40


In [48]:
insurance = Insurance.copy()
insurance_tiv_2015 = insurance.groupby("TIV_2015").agg({"PID":"count"}).rename(columns={"PID":"pd_num"}).reset_index()
same_insurance_value =insurance_tiv_2015[insurance_tiv_2015["pd_num"]>1]["TIV_2015"][0]

# find out unique location
insurance["location"]  = insurance["LAT"].astype(str)+ insurance["LON"].astype(str)
location_df = insurance.groupby("location").agg({"PID":"count"}).rename(columns={"PID":"location_num"}).reset_index()
unique_locations = location_df[location_df["location_num"]==1]["location"]

# filter out unique locations
mask = (insurance["location"].isin(unique_locations)) & (insurance["TIV_2015"]==same_insurance_value)
total_tiv_2016 = round(insurance[mask]["TIV_2016"].sum(),2)
pd.DataFrame({"TIV_2016":[total_tiv_2016]})

,TIV_2016
0,45


### Question 3: Human Traffic of Stadium

Write an SQL query to display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by visit_date in ascending order.

The query result format is in the following example.

`Stadium`

| id   | visit_date | people    |
|------|------------|-----------|
| 1    | 2017-01-01 | 10        |
| 2    | 2017-01-02 | 109       |
| 3    | 2017-01-03 | 150       |
| 4    | 2017-01-04 | 99        |
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |

`result`

| id   | visit_date | people    |
|------|------------|-----------|
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |



**Consecutive numbers can be found out by adding one column that ranks the number of people visiting the stadium**

In [51]:
Stadium = pd.DataFrame({"id":list(range(1,9)),
             "visit_date":["2017-01-01","2017-01-02","2017-01-03","2017-01-04","2017-01-05","2017-01-06",
                          "2017-01-07","2017-01-09"],
                       "people":[10,109,150,99,145,1455,199,188]})
Stadium

,id,visit_date,people
0,1,2017-01-01,10
1,2,2017-01-02,109
2,3,2017-01-03,150
3,4,2017-01-04,99
4,5,2017-01-05,145
5,6,2017-01-06,1455
6,7,2017-01-07,199
7,8,2017-01-09,188


In [83]:
stadium = Stadium.copy()
stadium_greater_than_100 = stadium[stadium["people"]>=100]
stadium_greater_than_100["rank"] = stadium_greater_than_100["id"].rank(method="first",ascending=True)
stadium_greater_than_100["flag"] = stadium_greater_than_100["id"] - stadium_greater_than_100["rank"]
stadium_greater_than_100["flag2"] = stadium_greater_than_100["id"].groupby(stadium_greater_than_100["flag"]).\
transform("count")
stadium_greater_than_100[stadium_greater_than_100["flag2"]>=3].iloc[:,:-3]

,id,visit_date,people
4,5,2017-01-05,145
5,6,2017-01-06,1455
6,7,2017-01-07,199
7,8,2017-01-09,188


## Question 4: Friend Requests II: Who Has the Most Friends

In social network like Facebook or Twitter, people send friend requests and accept others' requests as well.

`request_accepted`

| requester_id | accepter_id | accept_date|
|--------------|-------------|------------|
| 1            | 2           | 2016_06-03 |
| 1            | 3           | 2016-06-08 |
| 2            | 3           | 2016-06-08 |
| 3            | 4           | 2016-06-09 |

Write a query to find the the people who has most friends and the most friends number under the following rules:

It is guaranteed there is only 1 people having the most friends.
The friend request could only been accepted once, which mean there is no multiple records with the same requester_id and accepter_id value.
For the sample data above, the result is:



In [3]:
request_accepted = pd.DataFrame({"request_id":[1,1,2,3],
             "accepter_id":[2,3,3,4],
             "accept_date":["2016-06-03","2016-06-08","2016-06-08","2016-06-09"]})
request_accepted

,request_id,accepter_id,accept_date
0,1,2,2016-06-03
1,1,3,2016-06-08
2,2,3,2016-06-08
3,3,4,2016-06-09


In [7]:
trans_data = pd.concat([request_accepted["request_id"],request_accepted["accepter_id"]],
         axis=0).to_frame().rename(columns={0:"request_id"})
trans_data

,request_id
0,1
1,1
2,2
3,3
0,2
1,3
2,3
3,4


In [18]:
result_df = trans_data.groupby("request_id").agg({"request_id":"count"}).\
rename(columns={"request_id":"num"}).reset_index()
mask = result_df["num"]==result_df["num"].max()
result = result_df[mask]
result.rename(columns={"request_id":"id"})

,id,num
2,3,3


In [50]:
# method 2: order and select the first row
result_df.sort_values(by="num",ascending=False)

,request_id,num
2,3,3
0,1,2
1,2,2
3,4,1


## Question 5: Tree Nodes

| id | p_id |
|----|------|
| 1  | null |
| 2  | 1    |
| 3  | 1    |
| 4  | 2    |
| 5  | 2    |

Each node in the tree can be one of three types:
- Leaf: if the node is a leaf node.
- Root: if the node is the root of the tree.
- Inner: If the node is neither a leaf node nor a root node.
 

Write a query to print the node id and the type of the node. Sort your output by the node id. The result for the above sample is:



In [86]:
tree = pd.DataFrame({"id":[1,2,3,4,5],
             "p_id":[np.nan,1,1,2,2]})

def determine_type(x):
    """
    determine if a node is root, inner, or leaf
    """
    parent = tree[tree["id"]==x]["p_id"].values[0]
    
    if np.isnan(parent):
        return "Root"
    elif x in tree["p_id"].values:
        return "Inner"
    elif x not in tree["p_id"].values:
        return "Leaf"

tree["Type"] = tree["id"].apply(lambda x:determine_type(x))

In [87]:
tree[["id","Type"]]

,id,Type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


## Question 6: Shortest Distance in a Plane

Table point_2d holds the coordinates (x,y) of some unique points (more than two) in a plane.
 

Write a query to find the shortest distance between these points rounded to 2 decimals.

| x  | y  |
|----|----|
| -1 | -1 |
| 0  | 0  |
| -1 | -2 |

The shortest distance is 1.00 from point (-1,-1) to (-1,2). So the output should be:

| shortest |
|----------|
| 1.00     |

In [88]:
points_2d = pd.DataFrame({"x":[-1,0,-1],"y":[-1,0,-2]})
points_2d

,x,y
0,-1,-1
1,0,0
2,-1,-2


In [118]:
from itertools import permutations

merged_df = pd.merge(points_2d,points_2d,suffixes=("_t1","_t2"),how="cross")
mask = (merged_df["x_t1"]==merged_df["x_t2"]) & (merged_df["y_t1"]==merged_df["y_t2"])
trans_data = merged_df[~mask]
distances = np.sqrt((trans_data["x_t1"]-trans_data["x_t2"])**2 + (trans_data["y_t1"]-trans_data["y_t2"])**2)
pd.DataFrame({"shortest":[distances.min()]})

,shortest
0,1.0


## Question 6: Second Degree Follower

In facebook, there is a follow table with two columns: followee, follower.

Please write a sql query to get the amount of each follower’s follower if he/she has one.

| followee    | follower   |
|-------------|------------|
|     A       |     B      |
|     B       |     C      |
|     B       |     D      |
|     D       |     E      |



In [119]:
follow = pd.DataFrame({"followee":["A","B","B","D"],
             "follower":["B","C","D","E"]})
follow

,followee,follower
0,A,B
1,B,C
2,B,D
3,D,E


In [125]:
mask = follow["followee"].isin(follow["follower"])
result = follow[mask].groupby("followee").agg({"follower":"count"}).rename(columns={"follower":"num"}).reset_index()
result

,followee,num
0,B,2
1,D,1


## Question 7: Average Salary: Departments VS Company

Given two tables as below, write a query to display the comparison result (higher/lower/same) of the average salary of employees in a department to the company's average salary.
 
`Salary`

| id | employee_id | amount | pay_date   |
|----|-------------|--------|------------|
| 1  | 1           | 9000   | 2017-03-31 |
| 2  | 2           | 6000   | 2017-03-31 |
| 3  | 3           | 10000  | 2017-03-31 |
| 4  | 1           | 7000   | 2017-02-28 |
| 5  | 2           | 6000   | 2017-02-28 |
| 6  | 3           | 8000   | 2017-02-28 |

The employee_id column refers to the employee_id in the following table `employee`.

| employee_id | department_id |
|-------------|---------------|
| 1           | 1             |
| 2           | 2             |
| 3           | 2             |



In [153]:
# data preparation
dates = ["2017-03-31","2017-03-31","2017-03-31","2017-02-28","2017-02-28","2017-02-28"]
Salary = pd.DataFrame({"id":list(range(1,7)),
             "employee_id":[1,2,3,1,2,3],
             "amount":[9000,6000,10000,7000,6000,8000],
             "pay_date":pd.to_datetime(dates)})
employee = pd.DataFrame({"employee_id":[1,2,3],
                        "department_id":[1,2,2]})

In [155]:
Salary["pay_month"] = Salary["pay_date"].map(lambda x:datetime.strftime(x,"%Y-%m"))
Salary

,id,employee_id,amount,pay_date,pay_month
0,1,1,9000,2017-03-31,2017-03
1,2,2,6000,2017-03-31,2017-03
2,3,3,10000,2017-03-31,2017-03
3,4,1,7000,2017-02-28,2017-02
4,5,2,6000,2017-02-28,2017-02
5,6,3,8000,2017-02-28,2017-02


In [183]:
# we need to look up the department for each employee
sal_emp = pd.merge(Salary,employee,on="employee_id")
dept_avg_sal = sal_emp.groupby(["pay_month","department_id"]).agg({"amount":"mean"}).reset_index()
company_sal = sal_emp[["pay_month","amount"]].groupby("pay_month").agg({"amount":"mean"}).reset_index()
trans_data = pd.merge(dept_avg_sal,company_sal,on="pay_month",suffixes=("_dept","_comp"))

def isGreater(x):
    com_sal = trans_data[trans_data["amount_dept"]==x]["amount_comp"].values[0]
    if x>com_sal:
        return "higher"
    elif x==com_sal:
        return "same"
    else:
        return "lower"

trans_data["comparison"] = trans_data["amount_dept"].apply(lambda x:isGreater(x))
result = trans_data[["pay_month","department_id","comparison"]]
result

,pay_month,department_id,comparison
0,2017-02,1,same
1,2017-02,2,same
2,2017-03,1,higher
3,2017-03,2,lower


### Question 8:  Exchange Seats

Mary is a teacher in a middle school and she has a table seat storing students' names and their corresponding seat ids.

The column id is continuous increment.

Mary wants to change seats for the adjacent students.
If the number of students is odd, there is no need to change the last one's seat.


Can you write a SQL query to output the result for Mary?

`seat`

|    id   | student |
|---------|---------|
|    1    | Abbot   |
|    2    | Doris   |
|    3    | Emerson |
|    4    | Green   |
|    5    | Jeames  |

`result`

|    id   | student |
|---------|---------|
|    1    | Doris   |
|    2    | Abbot   |
|    3    | Green   |
|    4    | Emerson |
|    5    | Jeames  |



In [3]:
seat = pd.DataFrame({"id":np.arange(1,6),
             "student":["Abbot","Doris",'Emerson',"Green","Jeames"]})
seat

,id,student
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


**Solution**

- Create one column for new id
- Order by the new id, then the seat is rearranged

In [20]:
def change_seat(x):
    """change the id of seat student, from odd to even and even to odd"""
    if x%2==1 and x!=seat.id.max():
        x+=1
        return x
    elif x%2==1 and x==seat.id.max():
        return x
    elif x%2==0:
        x=x-1
        return x

seat["new_id"] = seat.id.apply(lambda x:change_seat(x))
new_seat = seat.sort_values(by="new_id",ascending=True).drop("id",axis=1)
res = new_seat.reindex(["new_id","student"],axis=1)
res

,new_id,student
1,1,Doris
0,2,Abbot
3,3,Green
2,4,Emerson
4,5,Jeames


### Question 9: Customers who bought all products

Write an SQL query for a report that provides the customer ids from the Customer table that bought all the products in the Product table.

Return the result table in any order.

The query result format is in the following example:

`Customer`

| customer_id | product_key |
|-------------|-------------|
| 1           | 5           |
| 2           | 6           |
| 3           | 5           |
| 3           | 6           |
| 1           | 6           |

Product_key is the foreign key of this table

`Product`

| product_key |
|-------------|
| 5           |
| 6           |

Product_key is the primary key of this table

`res`

| customer_id |
|-------------|
| 1           |
| 3           |

*Note: for customer table, product key is the foreign key that references product_key of product table, suggesting that the customer table might have duplicate records. In other words, one customer may buy the same product multiple times, so we need to drop duplicate records.*

In [21]:
# data preparation
Customer = pd.DataFrame({"customer_id":[1,2,3,3,1],
             "product_key":[5,6,5,6,6]})
Product = pd.DataFrame({"product_key":[5,6]})

In [44]:
# first we find out the total number of all products
max_prod_count = Product.product_key.count()

# then drop the duplicate records.
np_dup_customer = Customer.drop_duplicates(subset=["customer_id","product_key"])
customer_purchase = np_dup_customer.groupby("customer_id",as_index=False).agg({"product_key":"count"}).\
rename(columns={"product_key":"purchase_cnt"})
res = customer_purchase[customer_purchase["purchase_cnt"]==max_prod_count][["customer_id"]]
res

,customer_id
0,1
2,3


### Question 10: Product Sales Analysis III

Write an SQL query that selects the product id, year, quantity, and price for the first year of every product sold.

The query result format is in the following example:

`Sales`

| sale_id | product_id | year | quantity | price |
|---------|------------|------|----------|-------|
| 1       | 100        | 2008 | 10       | 5000  |
| 2       | 100        | 2009 | 12       | 5000  |
| 7       | 200        | 2011 | 15       | 9000  |

sale_id is the primary key of this table.
product_id is a foreign key to Product table.

`Product`

| product_id | product_name |
|------------|--------------|
| 100        | Nokia        |
| 200        | Apple        |
| 300        | Samsung      |

product_is is the foreign key that references product_id of sales table

`Result`

| product_id | first_year | quantity | price |
|------------|------------|----------|-------|
| 100        | 2008       | 10       | 5000  |
| 200        | 2011       | 15       | 9000  |



In [45]:
Sales = pd.DataFrame({"sale_id":[1,2,7],
             "product_id":[100,100,200],
             "year":[2008,2009,2011],
             "quantity":[10,12,15],
             "price":[5000,5000,9000]})
Product = pd.DataFrame({'product_id':[100,200,300],
                       "product_name":["Nokia","Apple","Samsung"]})

In [54]:
# first we need to find the first year of sales
first_year_sold = Sales.groupby("product_id",as_index=False).agg({"year":"min"})
res = pd.merge(first_year_sold,Sales,on=["product_id",'year'])
res

,product_id,year,sale_id,quantity,price
0,100,2008,1,10,5000
1,200,2011,7,15,9000


### Question 11: Project Employees III

Write an SQL query that reports the most experienced employees in each project. In case of a tie, report all employees with the maximum number of experience years.

The query result format is in the following example:

`Project`

| project_id  | employee_id |
|-------------|-------------|
| 1           | 1           |
| 1           | 2           |
| 1           | 3           |
| 2           | 1           |
| 2           | 4           |

`Employee`

| employee_id | name   | experience_years |
|-------------|--------|------------------|
| 1           | Khaled | 3                |
| 2           | Ali    | 2                |
| 3           | John   | 3                |
| 4           | Doe    | 2                |

`res`

| project_id  | employee_id   |
|-------------|---------------|
| 1           | 1             |
| 1           | 3             |
| 2           | 1             |



In [ ]:
# data preparation
Project = pd.DataFrame({"project_id":[1,1,1,2,2],
             "employee_id":[1,2,3,1,4]})
Employee = pd.DataFrame({"employee_id":[1,2,3,4],
                        "name":["Khaled","Ali","John","Doe"],
                        "experience_years":[3,2,3,2]})

In [69]:
# first we need to rank the experience years
merged_df = pd.merge(Project,Employee,on="employee_id")
merged_df["rank"] = merged_df["experience_years"].groupby(merged_df["project_id"]).rank(method="dense",ascending=False)
res = merged_df[merged_df["rank"] == 1][["project_id","employee_id"]].sort_values(by="project_id")
res

,project_id,employee_id
0,1,1
3,1,3
1,2,1


### Question 12: Sales Analysis II

Write an SQL query that reports the buyers who have bought S8 but not iPhone. Note that S8 and iPhone are products present in the Product table.



`Product`

| product_id | product_name | unit_price |
|------------|--------------|------------|
| 1          | S8           | 1000       |
| 2          | G4           | 800        |
| 3          | iPhone       | 1400       |

product_id is the primary key of this table.

`Sales`

| seller_id | product_id | buyer_id | sale_date  | quantity | price |
|-----------|------------|----------|------------|----------|-------|
| 1         | 1          | 1        | 2019-01-21 | 2        | 2000  |
| 1         | 2          | 2        | 2019-02-17 | 1        | 800   |
| 2         | 1          | 3        | 2019-06-02 | 1        | 800   |
| 3         | 3          | 3        | 2019-05-13 | 2        | 2800  |

This table does not have primary key, so it may have duplicate rows

`res`

| buyer_id    |
|-------------|
| 1           |

In [73]:
# data preparation
Product = pd.DataFrame({"product_id":[1,2,3],
             "product_name":["S8","G4","iphone"],
             "unit_price":[1000,800,1400]})
Sales = pd.DataFrame({"seller_id":[1,1,2,3],
                     "product_id":[1,2,1,3],
                     "buyer_id":[1,2,3,3],
                     "sale_date":pd.to_datetime(["2019-01-21","2019-02-17","2019-06-02","2019-05-13"]),
                     "quantity":[2,1,1,2],
                     "price":[2000,800,800,2800]})

In [83]:
# find out the product name to which each product corresponds
merged_df =pd.merge(Product,Sales,on="product_id")

# find out people who buy s8
s8_df = merged_df[merged_df.product_name=="S8"]
# find out people who buy iphone
iphone_df = merged_df[merged_df["product_name"]=="iphone"]

res = s8_df['buyer_id'][s8_df["buyer_id"].isin(iphone_df["buyer_id"])].to_frame()
res

,buyer_id
1,3


### Question 13: Sales Analysis III

Write an SQL query that reports the products that were only sold in spring 2019. That is, between 2019-01-01 and 2019-03-31 inclusive.

`Product`

| product_id | product_name | unit_price |
|------------|--------------|------------|
| 1          | S8           | 1000       |
| 2          | G4           | 800        |
| 3          | iPhone       | 1400       |

product_id is primary key of this table

`Sales`

| seller_id | product_id | buyer_id | sale_date  | quantity | price |
|-----------|------------|----------|------------|----------|-------|
| 1         | 1          | 1        | 2019-01-21 | 2        | 2000  |
| 1         | 2          | 2        | 2019-02-17 | 1        | 800   |
| 2         | 2          | 3        | 2019-06-02 | 1        | 800   |
| 3         | 3          | 4        | 2019-05-13 | 2        | 2800  |

This table does not have primary key, meaning duplicate rows

`res`

| product_id  | product_name |
|-------------|--------------|
| 1           | S8           |

The product with id 1 was only sold in spring 2019 while the other two were sold after.

In [99]:
Sales = pd.DataFrame({"seller_id":[1,1,2,3],
                     "product_id":[1,2,2,3],
                     "buyer_id":[1,2,3,3],
                     "sale_date":pd.to_datetime(["2019-01-21","2019-02-17","2019-06-02","2019-05-13"]),
                     "quantity":[2,1,1,2],
                     "price":[2000,800,800,2800]})

In [104]:
#### sales = Sales.copy()
sales["flag"] = sales["sale_date"].apply(lambda date:date >= datetime.strptime("2019-01-01","%Y-%m-%d")
                                        and date<=datetime.strptime("2019-03-31","%Y-%m-%d"))
sales_in_spring = sales.groupby("product_id",as_index=False).agg({"flag":"prod"})
is_Spring = sales_in_spring["flag"]== 1
merged_sales_pro = sales_in_spring[is_Spring].merge(Product,on="product_id")
res = merged_sales_pro[["product_id","product_name"]]
res

,product_id,product_name
0,1,S8


### Question 14: Game Play Analysis V

We define the install date of a player to be the first login day of that player.

We also define day 1 retention of some date X to be the number of players whose install date is X and they logged back in on the day right after X, divided by the number of players whose install date is X, rounded to 2 decimal places.

Write an SQL query that reports for each install date, the number of players that installed the game on that day and the day 1 retention.

`Activity`

| player_id | device_id | event_date | games_played |
|-----------|-----------|------------|--------------|
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-03-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-01 | 0            |
| 3         | 4         | 2016-07-03 | 5            |

`result`

| install_dt | installs | Day1_retention |
|------------|----------|----------------|
| 2016-03-01 | 2        | 0.50           |
| 2017-06-25 | 1        | 0.00           |


In [105]:
# data preparation
Activity = pd.DataFrame({"player_id":[1,1,2,3,3],
             "devide_id":[2,2,3,1,4],
             "event_date":pd.to_datetime(["2016-03-01","2016-03-02","2017-06-25","2016-03-01","2016-07-03"]),
             "games_played":[5,6,1,0,5]})

In [140]:
from pandas.tseries.offsets import Day

activity = Activity.copy()
activity["install_date"]  = activity["event_date"].groupby(activity["player_id"]).transform("min")

oneday = Day(1)
activity["is_log_back"] = activity['event_date'] - activity['install_date']==oneday

def get_nunique(col):
    return col.nunique()
grouped_res = activity.groupby("install_date",as_index=False).agg({"player_id":get_nunique,"is_log_back":"sum"}).\
rename(columns={"player_id":"installs"})
grouped_res['Day1_retention'] = grouped_res["is_log_back"]/grouped_res["installs"]
grouped_res[["install_date","installs","Day1_retention"]]

,install_date,installs,Day1_retention
0,2016-03-01,2,0.5
1,2017-06-25,1,0.0


### Question 15: Unpopular Books

Write an SQL query that reports the books that have sold less than 10 copies in the last year, excluding books that have been available for less than 1 month from today. Assume today is 2019-06-23.

The query result format is in the following example:

`Books`

| book_id | name               | available_from |
|---------|--------------------|----------------|
| 1       | "Kalila And Demna" | 2010-01-01     |
| 2       | "28 Letters"       | 2012-05-12     |
| 3       | "The Hobbit"       | 2019-06-10     |
| 4       | "13 Reasons Why"   | 2019-06-01     |
| 5       | "The Hunger Games" | 2008-09-21     |


`Orders`

| order_id | book_id | quantity | dispatch_date |
|----------|---------|----------|---------------|
| 1        | 1       | 2        | 2018-07-26    |
| 2        | 1       | 1        | 2018-11-05    |
| 3        | 3       | 8        | 2019-06-11    |
| 4        | 4       | 6        | 2019-06-05    |
| 5        | 4       | 5        | 2019-06-20    |
| 6        | 5       | 9        | 2009-02-02    |
| 7        | 5       | 8        | 2010-04-13    |

`res`

| book_id   | name               |
|-----------|--------------------|
| 1         | "Kalila And Demna" |
| 2         | "28 Letters"       |
| 5         | "The Hunger Games" |


In [141]:
Books = pd.DataFrame({"book_id":[1,2,3,4,5],
             "name":["Kalila And Demna","28 Letters","The Hobbit","13 Reasons Why","The Hunger Games"],
             "available_from":pd.to_datetime(["2010-01-01","2012-05-12","2019-06-10","2019-06-01","2008-09-21"])})
Orders = pd.DataFrame({"order_id":np.arange(1,8),
                      "book_id":[1,1,3,4,4,5,5],
                      "quantity":[2,1,8,6,5,9,8],
                      "dispatch_date":pd.to_datetime(["2018-07-26","2018-11-05","2019-06-11","2019-06-05","2019-06-20",
                                                     "2009-02-02","2010-04-13"])})

In [155]:
today = datetime(2019,6,23)
# first we find out books sold less than 10 copies last year
Is2018 = Orders["dispatch_date"].map(lambda x:x.year==2018)
orders_2018 = Orders[Is2018]
Lessthan10_books = orders_2018.groupby("book_id",as_index=False).agg({"quantity":"sum"})
Lessthan10_books = pd.merge(Books,Lessthan10_books,on="book_id",how='left').fillna(0)

# then filter out any books that is available from today less than 1 month
lessthan_1mon = today - Lessthan10_books["available_from"] < Day(30)
res = Lessthan10_books[~lessthan_1mon][["book_id","name"]]
res

,book_id,name
0,1,Kalila And Demna
1,2,28 Letters
4,5,The Hunger Games


### Question 16: New Users Daily Count

Write an SQL query that reports for every date within at most 90 days from today, the number of users that logged in for the first time on that date. Assume today is 2019-06-30.

`Traffic`

| user_id | activity | activity_date |
|---------|----------|---------------|
| 1       | login    | 2019-05-01    |
| 1       | homepage | 2019-05-01    |
| 1       | logout   | 2019-05-01    |
| 2       | login    | 2019-06-21    |
| 2       | logout   | 2019-06-21    |
| 3       | login    | 2019-01-01    |
| 3       | jobs     | 2019-01-01    |
| 3       | logout   | 2019-01-01    |
| 4       | login    | 2019-06-21    |
| 4       | groups   | 2019-06-21    |
| 4       | logout   | 2019-06-21    |
| 5       | login    | 2019-03-01    |
| 5       | logout   | 2019-03-01    |
| 5       | login    | 2019-06-21    |
| 5       | logout   | 2019-06-21    |

There is no primary key in this table

`res`

| login_date | user_count  |
|------------|-------------|
| 2019-05-01 | 1           |
| 2019-06-21 | 2           |



In [157]:
activity_date = pd.to_datetime(["2019-05-01"]*3+["2019-06-21"]*2+["2019-01-01"]*3+["2019-06-21"]*3+["2019-03-01"]*2+
                               ["2019-06-21"]*2)
Traffic = pd.DataFrame({"user_id":[1,1,1,2,2,3,3,3,4,4,4,5,5,5,5],
             "activity":["login","homepage","logout","login","logout","login","jobs","logout","login","groups","logout",
                        "login","logout","login","logout"],
                       "activity_date":activity_date})
Traffic

,user_id,activity,activity_date
0,1,login,2019-05-01
1,1,homepage,2019-05-01
2,1,logout,2019-05-01
3,2,login,2019-06-21
4,2,logout,2019-06-21
5,3,login,2019-01-01
6,3,jobs,2019-01-01
7,3,logout,2019-01-01
8,4,login,2019-06-21
9,4,groups,2019-06-21


In [172]:
# first find out the login date of each user
# copy the raw data
traffic = Traffic.copy()

# filter out login records and find out the login date
isLogin = traffic['activity']=="login"
login_df = traffic[isLogin]
login_df["login_date"] = login_df["activity_date"].groupby(login_df["user_id"]).transform("min")

# filter out dates that are wihtin 90 days from today
today = datetime(2019,6,30)
login_df["less_90"] = login_df["login_date"].apply(lambda date:today-date<Day(90))
less_than_90 = login_df["less_90"]==True
login_df_ = login_df[less_than_90]

# compute the number of users on that date
res = login_df_.groupby("login_date",as_index=False).agg({"user_id":"count"}).rename(columns={"user_id":"user_count"})
res

/Users/ray/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,login_date,user_count
0,2019-05-01,1
1,2019-06-21,2


### Question 17: Reported Posts II

Write an SQL query to find the average for daily percentage of posts that got removed after being reported as spam, rounded to 2 decimal places.

`actions`

| user_id | post_id | action_date | action | extra  |
|---------|---------|-------------|--------|--------|
| 1       | 1       | 2019-07-01  | view   | null   |
| 1       | 1       | 2019-07-01  | like   | null   |
| 1       | 1       | 2019-07-01  | share  | null   |
| 2       | 2       | 2019-07-04  | view   | null   |
| 2       | 2       | 2019-07-04  | report | spam   |
| 3       | 4       | 2019-07-04  | view   | null   |
| 3       | 4       | 2019-07-04  | report | spam   |
| 4       | 3       | 2019-07-02  | view   | null   |
| 4       | 3       | 2019-07-02  | report | spam   |
| 5       | 2       | 2019-07-03  | view   | null   |
| 5       | 2       | 2019-07-03  | report | racism |
| 5       | 5       | 2019-07-03  | view   | null   |
| 5       | 5       | 2019-07-03  | report | racism |

There is no primary key for this table, it may have duplicate rows.
The action column is an ENUM type of ('view', 'like', 'reaction', 'comment', 'report', 'share').
The extra column has optional information about the action such as a reason for report or a type of reaction. 

`removal`

| post_id | remove_date |
|---------|-------------|
| 2       | 2019-07-20  |
| 3       | 2019-07-18  |

post_id is the primary key of this table.
Each row in this table indicates that some post was removed as a result of being reported or as a result of an admin review.

`res`

| average_daily_percent |
|-----------------------|
| 75.00                 |

In [180]:
Actions = pd.DataFrame({"user_id":[1]*3+[2]*2+[3]*2+[4]*2+[5]*4,
             "post_id":[1]*3+[2]*2+[4]*2+[3]*2+[2]*2+[5]*2,
             "action_date":pd.to_datetime(["2019-07-01"]*3+["2019-07-04"]*4+["2019-07-02"]*2+["2019-07-03"]*4),
             "action":["view","like",'share']+["view",'report']*5,
             "extra":[np.nan]*4+["spam",np.nan]*3+["racism",np.nan,"racism"]})
Removal = pd.DataFrame({"post_id":[2,3],
                       "remove_date":pd.to_datetime(["2019-07-20","2019-07-18"])})

In [196]:
# first we need to find out emails reported as spam
actions = Actions.copy()
report_spam = (actions['action']=="report") & (actions["extra"] == "spam")
spam_emails = actions[report_spam]

# merge spam_emails with removal
merged_df = pd.merge(spam_emails,Removal,on="post_id",how='left')

# calculate the number of emails removed
remove_df = merged_df.groupby("action_date",as_index=False).agg({"remove_date":"count"})
# calculate the total number of emails reported as spam
post_cnt = merged_df.groupby("action_date",as_index=False).agg({"post_id":"count"}).rename(columns={'post_id':"post_count"})
# calculate the daily percentage
spam_df = pd.merge(remove_df,post_cnt)
spam_df["daily_percentage"] = spam_df["remove_date"] / spam_df["post_count"]
spam_df['daily_percentage'].mean()

0.75